In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn

#### Importing accounts_data_train.json

In [48]:
import json
with open("data/train/accounts_data_train.json", 'r') as f:
    accounts_data_train_json = json.load(f)

df_accounts_data_train = pd.DataFrame(columns=list( accounts_data_train_json[0][0].keys() ) + ['no_of_loan'])
df_accounts_data_train

accounts_data_train_json_new = []
for one_client in accounts_data_train_json:
    no_of_loans = len(one_client)
    for one_loan in one_client:
        one_loan['no_of_loans'] = no_of_loans
        accounts_data_train_json_new.append(one_loan)

accounts_data_train_json_new[:3]

df_accounts_data_train = pd.DataFrame.from_dict(accounts_data_train_json_new)
df_accounts_data_train = df_accounts_data_train[['uid', 'no_of_loans', 'credit_type', 'loan_amount', 'amount_overdue', 'open_date',
                                                'closed_date', 'payment_hist_string', ]]
print(df_accounts_data_train.shape)
df_accounts_data_train.head()

(1245310, 8)


,uid,no_of_loans,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string
0,AAA09044550,2,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,0000000000000000000000100000000000000000000000...
1,AAA09044550,2,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,000000000000000014044000000000000000000000000000
2,AAA10545297,1,Credit card,80996.445,0.0,2020-06-29,NaN,000000000000000000
3,AAA14112888,1,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,000000000
4,AAA20326915,8,Credit card,10480.500,0.0,2014-09-10,NaN,0000000000000000000000000000000000000000000000...


##### Check : checking if no_of_loans actually equals the value_count of uid

In [49]:
uid_with_my_count = df_accounts_data_train[['uid', 'no_of_loans']].set_index('uid')

counts = pd.DataFrame(df_accounts_data_train['uid'].value_counts())
counts.columns = ["counts"]
counts.index.names = ['uid']

for_count_check = counts.join(uid_with_my_count)

print("Zero indicates that our column noi_of_loans perfectly imitates the data")
for_count_check[for_count_check.counts != for_count_check.no_of_loans].shape[0]

Zero indicates that our column noi_of_loans perfectly imitates the data


0

#### Importing enquiry_data_train.json

In [50]:
with open("data/train/enquiry_data_train.json", 'r') as f:
    enquiry_data_train_json = json.load(f)

df_enquiry_data_train = pd.DataFrame(columns=list( enquiry_data_train_json[0][0].keys() ) + ['no_of_loan_enquiry'])
df_enquiry_data_train

enquiry_data_train_json_new = []
for one_client in enquiry_data_train_json:
    no_of_loans = len(one_client)
    for one_loan in one_client:
        one_loan['no_of_loans_enquiry'] = no_of_loans
        enquiry_data_train_json_new.append(one_loan)

enquiry_data_train_json_new[10:13]

df_enquiry_data_train = pd.DataFrame.from_dict(enquiry_data_train_json_new)
df_enquiry_data_train = df_enquiry_data_train[['uid', 'no_of_loans_enquiry', 'enquiry_type', 'enquiry_amt', 'enquiry_date', ]]
print(df_enquiry_data_train.shape)
df_enquiry_data_train.head()

(1909926, 5)


,uid,no_of_loans_enquiry,enquiry_type,enquiry_amt,enquiry_date
0,AAA08065248,11,Interbank credit,168839,2020-11-08
1,AAA08065248,11,Mobile operator loan,268392,2020-09-20
2,AAA08065248,11,Mobile operator loan,36082,2020-06-19
3,AAA08065248,11,Interbank credit,180467,2019-10-22
4,AAA08065248,11,Cash loan (non-earmarked),227459,2020-05-24


In [51]:
uid_with_my_count = df_enquiry_data_train[['uid', 'no_of_loans_enquiry']].set_index('uid')

counts = pd.DataFrame(df_enquiry_data_train['uid'].value_counts())
counts.columns = ["counts"]
counts.index.names = ['uid']

for_count_check = counts.join(uid_with_my_count)

print("Zero indicates that our column noi_of_loans perfectly imitates the data")
for_count_check[for_count_check.counts != for_count_check.no_of_loans_enquiry].shape[0]

Zero indicates that our column noi_of_loans perfectly imitates the data


0

#### Check : checking for Nan values

In [52]:
df_accounts_data_train.isna().sum()

uid                         0
no_of_loans                 0
credit_type                 0
loan_amount                 3
amount_overdue              0
open_date                   0
closed_date            463035
payment_hist_string         0
dtype: int64

In [53]:
df_enquiry_data_train.isna().sum()

uid                    0
no_of_loans_enquiry    0
enquiry_type           0
enquiry_amt            0
enquiry_date           0
dtype: int64

Only 3 missing in loan_amount, we can easily drop it, given that we have many train data 

closed_date null values indicates that the loan is still running

In [54]:
df_accounts_data_train = df_accounts_data_train[ df_accounts_data_train["loan_amount"].notna() ]

#### 

#### One hot encoding the enquiry type and credit type

In [55]:
def onehot_and_drop(df, feature):
    dummies = pd.get_dummies(df[[feature]])
    new = pd.concat([df, dummies], axis=1)
    new = new.drop([feature], axis=1)
    return new

categorical_features = ["credit_type"]
for feature in categorical_features:
    df_accounts_data_train = onehot_and_drop(df_accounts_data_train, feature)

categorical_features = ["enquiry_type"]
for feature in categorical_features:
    df_enquiry_data_train = onehot_and_drop(df_enquiry_data_train, feature)

#### Creating new column as 'loan_run'
* This column is difference between the open_date and closed_date
* Problem here is to decide on what to put value where there is nan value at closed, i.e still running 
* For now we have decided to give it day of today, assuming that this actually also tells us that the loan is still running, we would make a seperate column as loan closed to indicate that loan is still on

In [56]:
df_accounts_data_train["loan_closed"] = df_accounts_data_train["closed_date"].isnull().astype("int")
df_accounts_data_train['closed_date'].fillna("2022-12-29", inplace=True)


df_accounts_data_train['open_date'] = pd.to_datetime(df_accounts_data_train['open_date'])
df_accounts_data_train['closed_date'] = pd.to_datetime(df_accounts_data_train['closed_date'])
df_accounts_data_train["loan_run"] = df_accounts_data_train['closed_date'] - df_accounts_data_train['open_date']
df_accounts_data_train["loan_run"] = df_accounts_data_train["loan_run"].dt.days
df_accounts_data_train["loan_run"].describe()

count    1.245307e+06
mean     7.712967e+02
std      6.761992e+02
min     -3.968300e+04
25%      2.310000e+02
50%      5.750000e+02
75%      1.113000e+03
max      3.649000e+03
Name: loan_run, dtype: float64

#### Correcting where close date is before open date
There are only .0012 percentage data with this mistake, although we can guess that it was a human mistake of a swap. but we will drop these rows 
WE are still good without 13 data points

In [57]:
df_accounts_data_train = df_accounts_data_train[ df_accounts_data_train["loan_run"] > 0 ]
df_accounts_data_train["loan_run"].describe()

count    1.243812e+06
mean     7.722621e+02
std      6.750884e+02
min      1.000000e+00
25%      2.330000e+02
50%      5.780000e+02
75%      1.114000e+03
max      3.649000e+03
Name: loan_run, dtype: float64

In [58]:
df_accounts_data_train["loan_closed"] = df_accounts_data_train["loan_run"] > 0
df_accounts_data_train.head(3)

,uid,no_of_loans,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,credit_type_Another type of loan,credit_type_Car loan,credit_type_Cash loan (non-earmarked),...,credit_type_Loan for purchase of shares (margin lending),credit_type_Loan for the purchase of equipment,credit_type_Loan for working capital replenishment,credit_type_Microloan,credit_type_Mobile operator loan,credit_type_Mortgage,credit_type_Real estate loan,credit_type_Unknown type of loan,loan_closed,loan_run
0,AAA09044550,2,272745.000,0.0,2018-09-22,2020-02-22,0000000000000000000000100000000000000000000000...,0,0,0,...,0,0,0,0,0,0,0,0,True,518
1,AAA09044550,2,4500.000,0.0,2018-03-08,2019-07-25,000000000000000014044000000000000000000000000000,0,0,0,...,0,0,0,0,0,0,0,0,True,504
2,AAA10545297,1,80996.445,0.0,2020-06-29,2022-12-29,000000000000000000,0,0,0,...,0,0,0,0,0,0,0,0,True,913


## Handling Payment history string

    We would be creating 6 columns out of this for now, later on if need be, we would find out more
* No of times late

* Total days of late

* Maximum days of late

* Smallest Gap between late(Will be cosidered later)

* Largest Gap between late(Will be cosidered later)

* largest_consecutive_lates

* no_of_times_more_than_3_consecutive(Will be cosidered later)


####

In [59]:
def convert_to_payment_hist_list(payment_hist_string):

    length = len(payment_hist_string)

    payment_hist_list = []

    for i in range(0, length, 3):
        
        late_by_days = int(payment_hist_string[i:i+3])
        payment_hist_list.append(late_by_days)

    return payment_hist_list
    

df_accounts_data_train["payment_hist_list"] = df_accounts_data_train["payment_hist_string"].apply(convert_to_payment_hist_list)
df_accounts_data_train["payment_hist_list"].describe()

count                1243812
unique                 45212
top       [0, 0, 0, 0, 0, 0]
freq                  103179
Name: payment_hist_list, dtype: object

#### Total no of times client was late

In [60]:
def no_of_times_late(payment_hist_list_value):

    count = 0
    for this_month in payment_hist_list_value:
        if this_month > 0:
            count += 1

    return count

df_accounts_data_train["no_of_times_late"] = df_accounts_data_train["payment_hist_list"].apply(no_of_times_late)
df_accounts_data_train["no_of_times_late"].describe()

count    1.243812e+06
mean     2.816197e-01
std      1.599056e+00
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      8.100000e+01
Name: no_of_times_late, dtype: float64

#### Total no of lates days in client's history

In [61]:
def total_days_of_late(payment_hist_list_value):

    count = 0
    for this_month in payment_hist_list_value:
        if this_month > 0:
            count += this_month

    return count

df_accounts_data_train["total_late_days"] = df_accounts_data_train["payment_hist_list"].apply(total_days_of_late)
df_accounts_data_train["total_late_days"].describe()

count    1.243812e+06
mean     3.786600e+01
std      5.244029e+02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      4.224200e+04
Name: total_late_days, dtype: float64

#### Maximum no of days a client was late
That is what was the largest late run for a client till he paid

In [62]:
def maximum_late_days(payment_hist_list_value):

    max = -1
    for this_month in payment_hist_list_value:
        if this_month > max:
            max = this_month

    return max

df_accounts_data_train["max_late_days"] = df_accounts_data_train["payment_hist_list"].apply(maximum_late_days)
df_accounts_data_train["max_late_days"].describe()

count    1.243812e+06
mean     7.023836e+00
std      4.425434e+01
min     -1.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      9.990000e+02
Name: max_late_days, dtype: float64

#### Largest_consecutive_late

    Note that here consecutive has been counted as next months, ie [0,1,2,1,0,0] is 2 cosecutive, because next two months are consecutive along with first, not counted. 
    Such that client who was never late is -1, who were never two months in a row late is 0,

In [63]:
def largest_consecutive_late(payment_hist_list_value):
    
    count = -1
    max_consecutive_length = -1
    for this_month in payment_hist_list_value:
        if this_month > 0:
            count += 1
            if count > max_consecutive_length:
                max_consecutive_length = count
        else:
            count = -1

    return max_consecutive_length

df_accounts_data_train["largest_consecutive_late"] = df_accounts_data_train["payment_hist_list"].apply(largest_consecutive_late)

In [64]:
df_accounts_data_train.columns

Index(['uid', 'no_of_loans', 'loan_amount', 'amount_overdue', 'open_date',
       'closed_date', 'payment_hist_string',
       'credit_type_Another type of loan', 'credit_type_Car loan',
       'credit_type_Cash loan (non-earmarked)', 'credit_type_Consumer credit',
       'credit_type_Credit card', 'credit_type_Interbank credit',
       'credit_type_Loan for business development',
       'credit_type_Loan for purchase of shares (margin lending)',
       'credit_type_Loan for the purchase of equipment',
       'credit_type_Loan for working capital replenishment',
       'credit_type_Microloan', 'credit_type_Mobile operator loan',
       'credit_type_Mortgage', 'credit_type_Real estate loan',
       'credit_type_Unknown type of loan', 'loan_closed', 'loan_run',
       'payment_hist_list', 'no_of_times_late', 'total_late_days',
       'max_late_days', 'largest_consecutive_late'],
      dtype='object')

##### Making a seperate working copy of our data v1

In [65]:
version_1_account_data_train = df_accounts_data_train.drop(['open_date', 'closed_date', 'payment_hist_string', "payment_hist_list"], axis=1)
version_1_account_data_train.head(10)

,uid,no_of_loans,loan_amount,amount_overdue,credit_type_Another type of loan,credit_type_Car loan,credit_type_Cash loan (non-earmarked),credit_type_Consumer credit,credit_type_Credit card,credit_type_Interbank credit,...,credit_type_Mobile operator loan,credit_type_Mortgage,credit_type_Real estate loan,credit_type_Unknown type of loan,loan_closed,loan_run,no_of_times_late,total_late_days,max_late_days,largest_consecutive_late
0,AAA09044550,2,272745.000,0.0,0,0,0,1,0,0,...,0,0,0,0,True,518,1,10,10,0
1,AAA09044550,2,4500.000,0.0,0,0,0,1,0,0,...,0,0,0,0,True,504,2,58,44,1
2,AAA10545297,1,80996.445,0.0,0,0,0,0,1,0,...,0,0,0,0,True,913,0,0,0,-1
3,AAA14112888,1,43771.500,0.0,0,0,0,1,0,0,...,0,0,0,0,True,92,0,0,0,-1
4,AAA20326915,8,10480.500,0.0,0,0,0,0,1,0,...,0,0,0,0,True,3032,31,14477,917,30
5,AAA20326915,8,46696.500,0.0,0,0,0,1,0,0,...,0,0,0,0,True,320,0,0,0,-1
6,AAA20326915,8,0.000,0.0,0,0,0,0,1,0,...,0,0,0,0,True,1450,0,0,0,-1
7,AAA20326915,8,117000.000,0.0,0,0,0,1,0,0,...,0,0,0,0,True,91,0,0,0,-1
8,AAA20326915,8,90000.000,0.0,0,0,0,1,0,0,...,0,0,0,0,True,31,0,0,0,-1
9,AAA20326915,8,235800.000,0.0,0,0,0,1,0,0,...,0,0,0,0,True,45,0,0,0,-1


#### Grouping data

Grouping account data

In [66]:
dict_for_account_agg = {"no_of_loans" : 'first',
                        'loan_amount': ['sum', 'mean'],
                        'amount_overdue': ['sum', 'mean'],
                        "loan_run": ['sum', 'mean'], 
                        "loan_closed": 'sum',
                        "no_of_times_late": ['sum', 'mean'],
                        "total_late_days": ['sum', 'mean'],
                        "max_late_days": 'max',
                        "largest_consecutive_late": 'max'}

for column_name in version_1_account_data_train.columns:

    if column_name.startswith("credit_type"):
        dict_for_account_agg[column_name] = 'sum'

In [67]:
version_1_account_data_train.shape

(1243812, 25)

In [68]:
version_1_grouped_account_data_train = version_1_account_data_train.groupby("uid", as_index=False).agg(dict_for_account_agg)
version_1_grouped_account_data_train.columns = ['_'.join(col) for col in version_1_grouped_account_data_train.columns.values]
version_1_grouped_account_data_train.columns = [col.strip("_") for col in version_1_grouped_account_data_train.columns.values]
print(version_1_grouped_account_data_train.shape)
version_1_grouped_account_data_train.head(3)

(223903, 30)


,uid,no_of_loans_first,loan_amount_sum,loan_amount_mean,amount_overdue_sum,amount_overdue_mean,loan_run_sum,loan_run_mean,loan_closed_sum,no_of_times_late_sum,...,credit_type_Interbank credit_sum,credit_type_Loan for business development_sum,credit_type_Loan for purchase of shares (margin lending)_sum,credit_type_Loan for the purchase of equipment_sum,credit_type_Loan for working capital replenishment_sum,credit_type_Microloan_sum,credit_type_Mobile operator loan_sum,credit_type_Mortgage_sum,credit_type_Real estate loan_sum,credit_type_Unknown type of loan_sum
0,AAA09044550,2,277245.000,138622.500,0.0,0.0,1022,511.0,2,3,...,0,0,0,0,0,0,0,0,0,0
1,AAA10545297,1,80996.445,80996.445,0.0,0.0,913,913.0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,AAA14112888,1,43771.500,43771.500,0.0,0.0,92,92.0,1,0,...,0,0,0,0,0,0,0,0,0,0


grouping enquiry data

In [69]:
version_1_enquiry_data_train = df_enquiry_data_train.copy()

In [70]:
dict_for_enquiry_agg = {"no_of_loans_enquiry" : 'first',
                        'enquiry_amt': ['sum', 'mean'],
                        }

for column_name in version_1_enquiry_data_train.columns:

    if column_name.startswith("enquiry_type"):
        dict_for_enquiry_agg[column_name] = 'sum'

In [71]:
version_1_enquiry_data_train.shape

(1909926, 21)

In [72]:
version_1_grouped_enquiry_data_train = version_1_enquiry_data_train.groupby("uid", as_index=False).agg(dict_for_enquiry_agg)
version_1_grouped_enquiry_data_train.columns = ['_'.join(col) for col in version_1_grouped_enquiry_data_train.columns.values]
version_1_grouped_enquiry_data_train.columns = [col.strip("_") for col in version_1_grouped_enquiry_data_train.columns.values]
print(version_1_grouped_enquiry_data_train.shape)
version_1_grouped_enquiry_data_train.head(3)

(261383, 21)


,uid,no_of_loans_enquiry_first,enquiry_amt_sum,enquiry_amt_mean,enquiry_type_Another type of loan_sum,enquiry_type_Car loan_sum,enquiry_type_Cash loan (non-earmarked)_sum,enquiry_type_Cash loans_sum,enquiry_type_Consumer credit_sum,enquiry_type_Credit card_sum,...,enquiry_type_Loan for business development_sum,enquiry_type_Loan for purchase of shares (margin lending)_sum,enquiry_type_Loan for the purchase of equipment_sum,enquiry_type_Loan for working capital replenishment_sum,enquiry_type_Microloan_sum,enquiry_type_Mobile operator loan_sum,enquiry_type_Mortgage_sum,enquiry_type_Real estate loan_sum,enquiry_type_Revolving loans_sum,enquiry_type_Unknown type of loan_sum
0,AAA08065248,11,2064658,187696.181818,0,0,1,0,0,0,...,0,2,0,0,0,2,1,0,2,0
1,AAA09044550,26,2659000,102269.230769,3,3,1,7,1,1,...,0,0,1,0,2,1,1,1,1,1
2,AAA10545297,14,1317000,94071.428571,1,0,2,2,2,2,...,0,1,0,0,0,0,0,0,2,1


In [73]:
version_1_grouped_enquiry_data_train.shape, version_1_grouped_account_data_train.shape

((261383, 21), (223903, 30))

In [74]:
version_1_joined_grouped = pd.merge(version_1_grouped_account_data_train, version_1_grouped_enquiry_data_train, on='uid',how='left')
print(version_1_joined_grouped.shape)
version_1_joined_grouped.head()

(223903, 50)


,uid,no_of_loans_first,loan_amount_sum,loan_amount_mean,amount_overdue_sum,amount_overdue_mean,loan_run_sum,loan_run_mean,loan_closed_sum,no_of_times_late_sum,...,enquiry_type_Loan for business development_sum,enquiry_type_Loan for purchase of shares (margin lending)_sum,enquiry_type_Loan for the purchase of equipment_sum,enquiry_type_Loan for working capital replenishment_sum,enquiry_type_Microloan_sum,enquiry_type_Mobile operator loan_sum,enquiry_type_Mortgage_sum,enquiry_type_Real estate loan_sum,enquiry_type_Revolving loans_sum,enquiry_type_Unknown type of loan_sum
0,AAA09044550,2,277245.000,138622.500,0.0,0.0,1022,511.000,2,3,...,0,0,1,0,2,1,1,1,1,1
1,AAA10545297,1,80996.445,80996.445,0.0,0.0,913,913.000,1,0,...,0,1,0,0,0,0,0,0,2,1
2,AAA14112888,1,43771.500,43771.500,0.0,0.0,92,92.000,1,0,...,0,2,2,2,0,2,0,0,0,1
3,AAA20326915,8,591597.000,73949.625,0.0,0.0,6803,850.375,8,31,...,0,0,0,0,0,1,0,0,0,0
4,AAA31604840,5,1591960.500,318392.100,0.0,0.0,4600,920.000,5,0,...,3,2,2,0,0,0,0,2,0,4


#### Joining target columns according to uid

In [75]:
train_flag = pd.read_csv("data/train/train_flag.csv")
print(train_flag.shape)
train_flag.head(3)

(261383, 3)


,uid,NAME_CONTRACT_TYPE,TARGET
0,XDA69787158,Cash loans,0
1,BSE47789733,Cash loans,0
2,NTJ92213825,Cash loans,0


In [76]:
def onehot_and_drop(df, feature):
    dummies = pd.get_dummies(df[[feature]])
    new = pd.concat([df, dummies], axis=1)
    new = new.drop([feature], axis=1)
    return new

categorical_features = ["NAME_CONTRACT_TYPE"]
for feature in categorical_features:
    train_flag = onehot_and_drop(train_flag, feature)

train_flag.head()

,uid,TARGET,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans
0,XDA69787158,0,1,0
1,BSE47789733,0,1,0
2,NTJ92213825,0,1,0
3,TCQ47571695,0,1,0
4,WJZ68772744,0,1,0


In [77]:
version_1_joined_grouped = pd.merge(version_1_joined_grouped, train_flag, on='uid',how='left')
print(version_1_joined_grouped.shape)
version_1_joined_grouped.head()

(223903, 53)


,uid,no_of_loans_first,loan_amount_sum,loan_amount_mean,amount_overdue_sum,amount_overdue_mean,loan_run_sum,loan_run_mean,loan_closed_sum,no_of_times_late_sum,...,enquiry_type_Loan for working capital replenishment_sum,enquiry_type_Microloan_sum,enquiry_type_Mobile operator loan_sum,enquiry_type_Mortgage_sum,enquiry_type_Real estate loan_sum,enquiry_type_Revolving loans_sum,enquiry_type_Unknown type of loan_sum,TARGET,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans
0,AAA09044550,2,277245.000,138622.500,0.0,0.0,1022,511.000,2,3,...,0,2,1,1,1,1,1,0,1,0
1,AAA10545297,1,80996.445,80996.445,0.0,0.0,913,913.000,1,0,...,0,0,0,0,0,2,1,0,1,0
2,AAA14112888,1,43771.500,43771.500,0.0,0.0,92,92.000,1,0,...,2,0,2,0,0,0,1,0,1,0
3,AAA20326915,8,591597.000,73949.625,0.0,0.0,6803,850.375,8,31,...,0,0,1,0,0,0,0,0,1,0
4,AAA31604840,5,1591960.500,318392.100,0.0,0.0,4600,920.000,5,0,...,0,0,0,0,2,0,4,0,1,0


#### Final check for nulls

In [81]:
version_1_joined_grouped.isnull().sum().sum()

0

#### Saving our version 1 data
* saving in pickle
* saving in csv

In [82]:
version_1_joined_grouped.to_csv("version_1_joined_and_grouped.csv")

import pickle as pkl
with open("version_1_joined_and_grouped.pkl", "wb") as f:
    pkl.dump(version_1_joined_grouped, f)

In [83]:
version_1_joined_grouped.max_late_days_max.describe()

count    223903.000000
mean         34.639853
std          97.151084
min          -1.000000
25%           0.000000
50%           0.000000
75%          23.000000
max         999.000000
Name: max_late_days_max, dtype: float64